In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical

In [6]:
X_train = pd.read_csv(
    '/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/x_train_all.csv')

X_test = pd.read_csv(
    '/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/x_test_all.csv')

y_train = pd.read_csv(
    "/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/y_train_all.csv")

y_test = pd.read_csv(
    "/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/y_test_all.csv")

In [7]:
y_train['0'].value_counts()

0
2    2250
1    2220
4    1980
3    1410
8     540
6     360
9     270
7     240
0     210
5     210
Name: count, dtype: int64

In [8]:
from imblearn.over_sampling import SMOTE

# Create an instance of SMOTE
smote = SMOTE()

# Apply SMOTE to the training data
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Check the class distribution after oversampling
print(y_train_resampled['0'].value_counts())


0
0    2250
1    2250
2    2250
3    2250
4    2250
5    2250
6    2250
7    2250
8    2250
9    2250
Name: count, dtype: int64


In [9]:
image_height = 48
image_width = 48
num_channels = 1
num_classes = 10
num_epochs = 10
batch_size = 64

In [10]:
# Reshape X_train to match the expected input shape
X_train_reshaped = np.reshape(
    X_train, (X_train.shape[0], image_height, image_width, num_channels))

# Reshape X_test to match the expected input shape
X_test_reshaped = np.reshape(
    X_test, (X_test.shape[0], image_height, image_width, num_channels))

# Convert y_train to one-hot encoded format
y_train_encoded = to_categorical(y_train, num_classes)

# Convert y_test to one-hot encoded format
y_test_encoded = to_categorical(y_test, num_classes)

In [11]:
from keras.layers import Dropout

# 98% accuracy on the test set

# Define the CNN model with dropout layers
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(image_height, image_width, num_channels)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Add dropout layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Add dropout layer
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Add dropout layer
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  # Add dropout layer
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))  # Add dropout layer
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped, y_train_encoded, epochs=num_epochs, batch_size=batch_size)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-11-23 12:58:20.090181: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-23 12:58:20.090325: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/10


2023-11-23 12:58:21.031675: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-11-23 12:58:21.379485: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


152/152 [==============================] - 9s 31ms/step - loss: 2.9428 - accuracy: 0.2711
Epoch 2/10
152/152 [==============================] - 4s 25ms/step - loss: 1.2690 - accuracy: 0.5429
Epoch 3/10
152/152 [==============================] - 4s 25ms/step - loss: 0.5096 - accuracy: 0.8497
Epoch 4/10
152/152 [==============================] - 4s 25ms/step - loss: 0.2588 - accuracy: 0.9305
Epoch 5/10
152/152 [==============================] - 4s 25ms/step - loss: 0.1774 - accuracy: 0.9508
Epoch 6/10
152/152 [==============================] - 4s 25ms/step - loss: 0.1398 - accuracy: 0.9633
Epoch 7/10
152/152 [==============================] - 4s 25ms/step - loss: 0.0887 - accuracy: 0.9741
Epoch 8/10
152/152 [==============================] - 4s 25ms/step - loss: 0.1016 - accuracy: 0.9720
Epoch 9/10
152/152 [==============================] - 4s 26ms/step - loss: 0.0684 - accuracy: 0.9802
Epoch 10/10
152/152 [==============================] - 4s 26ms/step - loss: 0.0786 - accuracy: 0.9785


In [12]:
model.evaluate(X_train_reshaped, y_train_encoded)

  5/303 [..............................] - ETA: 4s - loss: 0.0377 - accuracy: 0.9812      

2023-11-23 12:59:04.569758: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


303/303 [==============================] - 4s 11ms/step - loss: 0.0098 - accuracy: 0.9979


[0.009827748872339725, 0.9979360103607178]

In [13]:
model.evaluate(X_test_reshaped, y_test_encoded)

97/97 [==============================] - 1s 12ms/step - loss: 0.0437 - accuracy: 0.9861


[0.04366050660610199, 0.9860841631889343]

### Here we got 98.6% accuracy on the test set due to producing synthetic data.